In [3]:
!pip install torch transformers scikit-learn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigcode/starcoder"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)
model

In [111]:
def tokenize_codebert(code: str, summary: str = None):
    code_tokens = tokenizer(code, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
    # if summary:
    #     nl_tokens=tokenizer.tokenize(summary)
    #     tokens = [tokenizer.cls_token] + nl_tokens + [tokenizer.sep_token]
    # tokens += code_tokens + [tokenizer.eos_token]
    tokens_ids = tokenizer.convert_tokens_to_ids(code_tokens)
    context_embeddings = model(torch.tensor(tokens_ids)[None,:])[0]
    return context_embeddings

token: torch.Tensor = tokenize_codebert("if a>b: max=a else max=b", "store maximum value")
token.detach().numpy().shape

(1, 2, 768)

In [103]:
import sys
sys.path.append("../")
import json
from utils.helper_functions import clean_code


# Read the JSON file
with open('../../data/final_test.json') as f:
    data = json.load(f)

# Print the JSON object
X = [clean_code(cell["source"]) for cell in data["cells"]]
X

["import numpy as np \nimport pandas as pd \n\n\n\n\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))",
 "df = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')",
 "import numpy as np \nimport pandas as pd \n\n\n\n\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))",
 'import numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns',
 "train=pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')\ntest=pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')",
 'df=pd.concat([train,test])',
 'df.head()',
 'df.shape',
 'df.info()',
 'df.isnull().sum()',
 'df.describe()',
 'df.corr(numeric_only=True).abs()',
 "plt.figure(figsize=(12, 6))\nsns.barplot(x='MonsoonIntensity', y='FloodProbability', data=train)\nplt.title('Monsoon Intensity vs Flood P

In [104]:
embed_X_codebert = [tokenize_codebert(x).detach().numpy() for x in X]
embed_X_codebert

[array([[[-0.10990553,  0.3639664 ,  0.08284222, ..., -0.23133305,
          -0.32254973,  0.32922897],
         [-0.10424355,  0.39672735,  0.32969233, ..., -0.6600961 ,
          -0.2580749 ,  0.22606501]]], dtype=float32),
 array([[[-0.10990553,  0.3639664 ,  0.08284222, ..., -0.23133305,
          -0.32254973,  0.32922897],
         [-0.10424355,  0.39672735,  0.32969233, ..., -0.6600961 ,
          -0.2580749 ,  0.22606501]]], dtype=float32),
 array([[[-0.10990553,  0.3639664 ,  0.08284222, ..., -0.23133305,
          -0.32254973,  0.32922897],
         [-0.10424355,  0.39672735,  0.32969233, ..., -0.6600961 ,
          -0.2580749 ,  0.22606501]]], dtype=float32),
 array([[[-0.10990553,  0.3639664 ,  0.08284222, ..., -0.23133305,
          -0.32254973,  0.32922897],
         [-0.10424355,  0.39672735,  0.32969233, ..., -0.6600961 ,
          -0.2580749 ,  0.22606501]]], dtype=float32),
 array([[[-0.10990553,  0.3639664 ,  0.08284222, ..., -0.23133305,
          -0.32254973,  0.329

In [95]:
from sklearn.cluster import HDBSCAN

clusterer: HDBSCAN = HDBSCAN(
    min_cluster_size=3,                 # Minimum number of samples to form a cluster
    min_samples=5,                      # Minimum number of samples in a neighborhood to be considered as a core point
    cluster_selection_epsilon= 0,     # If 2 clusters are less than epsilon apart, they get merged
)

clusterer.fit(embed_X_codebert)
len(list(filter(lambda x: x == -1, clusterer.labels_)))

1080

In [109]:
embed_X_tfidf = tokenize_tfidf(X)
np.array(embed_X_tfidf)

array(<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14188 stored elements and shape (1080, 2357)>, dtype=object)